In [ ]:
%cd /content/drive/MyDrive/Colab/Trial

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img, ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import shutil
import math
from IPython.display import Image as IPImage
from IPython.display import display_jpeg

print(tf.__version__)

train_dir = 'target_datasets/train'
val_dir = 'target_datasets/val'

backup_dir = './model'

labels = [d for d in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, d))]
labels.sort()

2.7.0


In [3]:
#シードの設定
seed=1234
initializer=tf.keras.initializers.GlorotUniform(seed=seed)

NUM_CLASSES = len(labels)
print("ラベル数："+str(NUM_CLASSES))
# 学習率
LEARNING_RATE = 0.0005
# エポック（世代数）
EPOCHS = 24
# バッチサイズ
BATCH_SIZE = 16

IMAGE_SIZE = 256

train_data_gen = ImageDataGenerator(rescale=1./255,
                                    width_shift_range=0.1,
                                    height_shift_range=0.1,
                                    zoom_range=0.1,
                                    horizontal_flip=True,)
val_data_gen = ImageDataGenerator(rescale=1./255)

train_data = train_data_gen.flow_from_directory(
    train_dir, target_size=(IMAGE_SIZE, IMAGE_SIZE),
    color_mode='rgb', batch_size=BATCH_SIZE,
    class_mode='categorical', shuffle=True)

validation_data = val_data_gen.flow_from_directory(
    val_dir, target_size=(IMAGE_SIZE, IMAGE_SIZE),
    color_mode='rgb', batch_size=BATCH_SIZE,
    class_mode='categorical', shuffle=False)

ラベル数：7
Found 296 images belonging to 7 classes.
Found 76 images belonging to 7 classes.


In [ ]:
model = Sequential()

model.add(Conv2D(16, (3, 3), padding='same',
    input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('relu'))

model.add(Conv2D(16, (3, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('relu'))

model.add(Conv2D(16, (3, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('relu'))

model.add(Conv2D(16, (3, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('relu'))

model.add(Dropout(rate=0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))

model.add(Dropout(rate=0.2))

model.add(Dense(NUM_CLASSES))
model.add(Activation('softmax'))

opt = tf.keras.optimizers.Adam(lr=LEARNING_RATE)

model.compile(opt, loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

# Execute train
history = model.fit(train_data, epochs=EPOCHS, validation_data=validation_data, verbose=1)

score = model.evaluate(validation_data)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# Save model
save_model_path = os.path.join(backup_dir, 'model_1221.h5')
model.save(save_model_path)

In [ ]:
%matplotlib inline
from PIL import Image
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def get_category(x):
  if len(x)!=7:
    return 0
  max_index = np.argmax(x)
  cat_list=["anime","building","document","food","mono","person","snowscene"]

  return cat_list[max_index]

img_li=[]
files = glob.glob("predict_data/*")
for i,img_name in enumerate(files):
  im = tf.io.read_file(img_name)
  im = tf.image.decode_jpeg(im, channels=3)
  #画像のリサイズ
  im=tf.image.resize(im, [IMAGE_SIZE, IMAGE_SIZE])/255
  img_li.append(im)
img_li=np.array(img_li)

model_path = os.path.join(backup_dir, 'model_1221.h5')
new_model=tf.keras.models.load_model(model_path)
results=new_model.predict(img_li)

for i in range(len(results)):
  img_name=files[i]
  print(img_name)
  img = mpimg.imread(img_name)
  imgplot = plt.imshow(img)
  plt.show()
  print(get_category(results[i]))